Load packages

In [ ]:
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
from scipy import fftpack
import pandas as pd
from pathlib import Path
import os
import copy
import json
from IPython.display import display
from ipyfilechooser import FileChooser
from ephyviewer import mkQApp, MainViewer, TraceViewer
from ephyviewer import AnalogSignalSourceWithScatter
import ephyviewer
from scipy.stats import zscore
from scipy.interpolate import interp1d
from itertools import groupby
import sys 
import pickle
import seaborn as sns
from scipy.signal import find_peaks
from scipy.signal import chirp, find_peaks, peak_widths
from scipy import stats
import matplotlib.colors as mcolors
import warnings
from scipy import interpolate
import re
from scipy.stats import f_oneway
from scipy.spatial.distance import pdist, squareform
from scipy.stats import mannwhitneyu
from scipy.stats import ks_2samp
import diptest
import random
from collections import Counter, defaultdict
import ast
warnings.filterwarnings("ignore")

#%matplotlib widget

### Correlation across vigilance states

In [ ]:
try: # tries to retrieve dfilepath either from a previous run or from a previous notebook
    %store -r dfilepath
except:
    print("the path was not defined in store")
    #dpath = "/Users/mb/Documents/Syntuitio/AudreyHay/PlanB/ExampleRedLines/2022_08_06/13_30_01/My_V4_Miniscope/"
dfilepath = "//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/"

fc1 = FileChooser(dfilepath,select_default=True, show_only_dirs = False, title = "<b>Load excel or pkl file</b>")
display(fc1)

# Sample callback function
def update_my_folder(chooser):
    global dfilepath
    dfilepath = chooser.selected
    %store dfilepath
    return 

# Register callback function
fc1.register_callback(update_my_folder)

In [ ]:
try :
    combined_df = pd.read_excel(dfilepath, index_col=0 , sheet_name=None)
except:
    with open(dfilepath, 'rb') as pickle_file:
        combined_df = pickle.load(pickle_file)
        
# dfilepath="//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/Analysed2025_AB/_global_analysis/AVG_VigSt_2025-04-17_15_22_48_TEST/Baseline/Cluster0units/L1NDNF_mice_VigSt_PairCorrCa_Cluster0units.xlsx"

fig, axes = plt.subplots(1, 4, figsize=(15, 5))
VMAX=1
VMIN=-.2

CaCorrMatrix=combined_df['baseline_AW']
df = CaCorrMatrix.apply(pd.to_numeric, errors='coerce')
ax = sns.heatmap(df, ax=axes[0], square=True, vmin=VMIN , vmax=VMAX , cmap='viridis',
                                  cbar_kws={'shrink': 0.6, 'aspect': 20})
# Set only the first and last ticks for x and y axes
# Get the number of ticks
x_ticks = ax.get_xticks()
y_ticks = ax.get_yticks()
ax.set_xticks([x_ticks[0], x_ticks[-1]])
ax.set_yticks([y_ticks[0], y_ticks[-1]])

# Optionally, set the labels for these ticks if needed
ax.set_xticklabels([f'#{int(x_ticks[0])+1}', f'#{int(x_ticks[-1])+1}'],size=6)
ax.set_yticklabels([f'#{int(y_ticks[0])+1}', f'#{int(y_ticks[-1])+1}'],size=6)

# Optionally, add labels for the axes if needed
ax.set_xlabel('Neurons', labelpad=-2,size=8) 
ax.set_ylabel('Neurons', labelpad=-8,size=8) 
cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=6)  # Adjust the font size here
cbar.set_label('Correlation coefficient (r)', fontsize=8, font='Arial', rotation=-90, labelpad=10)
ax.xaxis.set_ticks_position('top')  # Move ticks to the top
ax.xaxis.set_label_position('top')  # Move label to the top
axes[0].set_title(f'AW (r={(np.round(np.nanmean(CaCorrMatrix),2))})', fontsize=12)


CaCorrMatrix=combined_df['baseline_QW']
df = CaCorrMatrix.apply(pd.to_numeric, errors='coerce')
ax = sns.heatmap(df, ax=axes[1], square=True, vmin=VMIN , vmax=VMAX , cmap='viridis',
                                  cbar_kws={'shrink': 0.6, 'aspect': 20})
# Set only the first and last ticks for x and y axes
# Get the number of ticks
x_ticks = ax.get_xticks()
y_ticks = ax.get_yticks()
ax.set_xticks([x_ticks[0], x_ticks[-1]])
ax.set_yticks([y_ticks[0], y_ticks[-1]])

# Optionally, set the labels for these ticks if needed
ax.set_xticklabels([f'#{int(x_ticks[0])+1}', f'#{int(x_ticks[-1])+1}'],size=6)
ax.set_yticklabels([f'#{int(y_ticks[0])+1}', f'#{int(y_ticks[-1])+1}'],size=6)

# Optionally, add labels for the axes if needed
ax.set_xlabel('Neurons', labelpad=-2,size=8) 
ax.set_ylabel('Neurons', labelpad=-8,size=8) 
cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=6)  # Adjust the font size here
cbar.set_label('Correlation coefficient (r)', fontsize=8, font='Arial', rotation=-90, labelpad=10)
ax.xaxis.set_ticks_position('top')  # Move ticks to the top
ax.xaxis.set_label_position('top')  # Move label to the top
axes[1].set_title(f'QW (r={(np.round(np.nanmean(CaCorrMatrix),2))})', fontsize=12)


CaCorrMatrix=combined_df['baseline_NREM']
df = CaCorrMatrix.apply(pd.to_numeric, errors='coerce')
ax = sns.heatmap(df, ax=axes[2], square=True, vmin=VMIN , vmax=VMAX , cmap='viridis',
                                  cbar_kws={'shrink': 0.6, 'aspect': 20})
# Set only the first and last ticks for x and y axes
# Get the number of ticks
x_ticks = ax.get_xticks()
y_ticks = ax.get_yticks()
ax.set_xticks([x_ticks[0], x_ticks[-1]])
ax.set_yticks([y_ticks[0], y_ticks[-1]])

# Optionally, set the labels for these ticks if needed
ax.set_xticklabels([f'#{int(x_ticks[0])+1}', f'#{int(x_ticks[-1])+1}'],size=6)
ax.set_yticklabels([f'#{int(y_ticks[0])+1}', f'#{int(y_ticks[-1])+1}'],size=6)

# Optionally, add labels for the axes if needed
ax.set_xlabel('Neurons', labelpad=-2,size=8) 
ax.set_ylabel('Neurons', labelpad=-8,size=8) 
cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=6)  # Adjust the font size here
cbar.set_label('Correlation coefficient (r)', fontsize=8, font='Arial', rotation=-90, labelpad=10)
ax.xaxis.set_ticks_position('top')  # Move ticks to the top
ax.xaxis.set_label_position('top')  # Move label to the top
axes[2].set_title(f'NREM (r={(np.round(np.nanmean(CaCorrMatrix),2))})', fontsize=12)



CaCorrMatrix=combined_df['baseline_REM']
df = CaCorrMatrix.apply(pd.to_numeric, errors='coerce')
ax = sns.heatmap(df, ax=axes[3], square=True, vmin=VMIN , vmax=VMAX , cmap='viridis',
                                  cbar_kws={'shrink': 0.6, 'aspect': 20})
# Set only the first and last ticks for x and y axes
# Get the number of ticks
x_ticks = ax.get_xticks()
y_ticks = ax.get_yticks()
ax.set_xticks([x_ticks[0], x_ticks[-1]])
ax.set_yticks([y_ticks[0], y_ticks[-1]])

# Optionally, set the labels for these ticks if needed
ax.set_xticklabels([f'#{int(x_ticks[0])+1}', f'#{int(x_ticks[-1])+1}'],size=6)
ax.set_yticklabels([f'#{int(y_ticks[0])+1}', f'#{int(y_ticks[-1])+1}'],size=6)

# Optionally, add labels for the axes if needed
ax.set_xlabel('Neurons', labelpad=-2,size=8) 
ax.set_ylabel('Neurons', labelpad=-8,size=8) 
cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=6)  # Adjust the font size here
cbar.set_label('Correlation coefficient (r)', fontsize=8, font='Arial', rotation=-90, labelpad=10)
ax.xaxis.set_ticks_position('top')  # Move ticks to the top
ax.xaxis.set_label_position('top')  # Move label to the top
axes[3].set_title(f'REM (r={np.round(np.nanmean(CaCorrMatrix),2)})', fontsize=12)


#plt.savefig(f'C:/Users/Manip2/Documents/ElifePaper/Rawdata/Extract{mice}_{folder_base.parts[-2]}_3VigSt_{beg}to{fin}s_2heatmap.svg', format='svg', bbox_inches='tight', pad_inches=0, transparent=True)
plt.tight_layout()
plt.show()

### Cell assemblies activity across vigilance states

In [ ]:
try: # tries to retrieve dfilepath either from a previous run or from a previous notebook
    %store -r dfilepath
except:
    print("the path was not defined in store")
    #dpath = "/Users/mb/Documents/Syntuitio/AudreyHay/PlanB/ExampleRedLines/2022_08_06/13_30_01/My_V4_Miniscope/"
    dfilepath = "//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/"

fc1 = FileChooser(Path(dfilepath).parent, select_default=True, show_only_dirs = False, title = "<b>Load excel file</b>")
display(fc1)

# Sample callback function
def update_my_folder(chooser):
    global dfilepath
    dfilepath = chooser.selected
    %store dfilepath
    return 

# Register callback function
fc1.register_callback(update_my_folder)

Plot Cell assembly activity relative to Vig States

In [ ]:
try :
    combined_df0 = pd.read_excel(dfilepath, index_col=0)
except:
    with open(dfilepath, 'rb') as pickle_file:
        combined_df0 = pickle.load(pickle_file)
#dfilepath="//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/Analysed2025_AB/_baseline_analysis/VigSt_2025-04-21_09_46_24_noCorr/CellAssembly_Global.xlsx"

combined_df00=combined_df0.copy()
combined_df00['Cells_in_Assembly']=combined_df00['Cells_in_Assembly'] = combined_df00['Cells_in_Assembly'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
combined_df00 = combined_df00[combined_df00['Cells_in_Assembly'].apply(len) == combined_df00['Assembly_size']]
try :
    combined_df00.to_excel(dfilepath)
except:
    with open(dfilepath, 'wb') as pickle_file:
        pickle.dump(combined_df00, pickle_file)

#combined_df00 = combined_df00[combined_df00['ExpeType'] == 'baseline']

NrSubtype='L2_3_mice' # L1NDNF_mice OR L2_3_mice
combined_df= combined_df00[combined_df00['NeuronType']==NrSubtype]
plt.figure(figsize=(6, 6))
combined_df2 = combined_df.pivot_table(index='Assembly_ID', columns=[combined_df['Substate']], values='Avg_Activity', aggfunc='mean', fill_value=None)
desired_order = ['AW','QW', 'NREM', 'IS', 'REM', 'undefined']   
try: combined_df2 = combined_df2[desired_order]
except: pass
try:del combined_df2['undefined']
except: pass
try:del combined_df2['IS']
except: pass
plt.subplot(2,2,1)
plt.plot(combined_df2.columns, combined_df2.values.T, alpha=0.5, linewidth=2)
plt.ylabel('Avg activity per cell assemblies')
plt.title(f'{NrSubtype}, n={len(combined_df2)}')
plt.tight_layout()

combined_df2 = combined_df.pivot_table(index='Assembly_ID', columns=[combined_df['Substate']], values='EventFreq', aggfunc='mean', fill_value=None)
try: combined_df2 = combined_df2[desired_order]
except: pass
try:del combined_df2['undefined']
except: pass
try:del combined_df2['IS']
except: pass
plt.subplot(2,2,3)
plt.plot(combined_df2.columns, combined_df2.values.T, alpha=0.5, linewidth=2)
plt.ylabel('Event frequency per cell assemblies')

NrSubtype='L1NDNF_mice' # L1NDNF_mice OR L2_3_mice
combined_df= combined_df00[combined_df00['NeuronType']==NrSubtype]
combined_df2 = combined_df.pivot_table(index='Assembly_ID', columns=[combined_df['Substate']], values='Avg_Activity', aggfunc='mean', fill_value=None)
desired_order = ['AW','QW', 'NREM', 'IS', 'REM', 'undefined']   
try: combined_df2 = combined_df2[desired_order]
except: pass
try:del combined_df2['undefined']
except: pass
try:del combined_df2['IS']
except: pass
plt.subplot(2,2,2)
plt.plot(combined_df2.columns, combined_df2.values.T, alpha=0.5, linewidth=2)
plt.ylabel('Avg activity per cell assemblies')
plt.title(f'{NrSubtype}, n={len(combined_df2)}')
plt.tight_layout()

combined_df2 = combined_df.pivot_table(index='Assembly_ID', columns=[combined_df['Substate']], values='EventFreq', aggfunc='mean', fill_value=None)
try: combined_df2 = combined_df2[desired_order]
except: pass
try:del combined_df2['undefined']
except: pass
try:del combined_df2['IS']
except: pass
plt.subplot(2,2,4)
plt.plot(combined_df2.columns, combined_df2.values.T, alpha=0.5, linewidth=2)
plt.ylabel('Event frequency per cell assemblies')
plt.tight_layout()

plt.show()

### Cell assembly identity

Load VigSt_Global_cluster file

In [ ]:
with open(f'{Path(dfilepath).parent}/VigStates_Global_cluster.pkl', 'rb') as pickle_file:
    df_cluster = pickle.load(pickle_file)

df_cluster = df_cluster[df_cluster['NeuronType'] == 'L1NDNF_mice']
df_cluster_Drug = df_cluster.copy()

#df_cluster_Drug = df_cluster_Drug[df_cluster_Drug['Drug'] == 'baseline']

AllBaselineUnits = df_cluster_Drug['Unit_ID'].unique()
Cluster0units = df_cluster_Drug[df_cluster_Drug['ClusterHDBSCAN'] == 0]['Unit_ID'].unique()
Cluster1units = df_cluster_Drug[df_cluster_Drug['ClusterHDBSCAN'] == 1]['Unit_ID'].unique()
Cluster2units = df_cluster_Drug[df_cluster_Drug['ClusterHDBSCAN'] == 2]['Unit_ID'].unique()

Identify Cell assembly Cluster ID and test proportion

In [ ]:
# --- Setup: Individuals and clusters
new_df2 = combined_df.drop_duplicates(subset='Assembly_ID', keep='first')
new_df2 = new_df2[new_df2['Cells_in_Assembly'].apply(len) == new_df2['Assembly_size']]
groups = new_df2['Cells_in_Assembly'].tolist()
df_unique = df_cluster_Drug.drop_duplicates(subset='Unit_ID', keep='first')
df_unique = df_unique.dropna(subset=['ClusterHDBSCAN'])
ids=df_unique['Unit_ID'].tolist()
id_to_cluster = dict(zip(df_unique['Unit_ID'], np.floor(df_unique['ClusterHDBSCAN']).astype(str)))
cluster_labels = ['0.0']*len(Cluster0units) + ['1.0']*len(Cluster1units) + ['2.0']*len(Cluster2units)
#cluster_labels = new_df2_c['Assembly_Cluster_ID'].tolist()

def classify_groups(groups, id_to_cluster):
    labels = []
    for group in groups:
        clusters = [id_to_cluster.get(i, np.nan) for i in group]
        if len(clusters) >= 1:
            count = Counter(clusters)
            top, n_top = count.most_common(1)[0]
            top_clusters = [k for k, v in count.items() if v == n_top]        
            # If there are multiple top clusters or the majority is less than 50%, label as "mixte"
            if len(top_clusters) == 1 and (n_top / len(group) * 100) > 50 and not pd.isna(top):
                labels.append(top)
            else:
                labels.append("mixte")  
        else:
            labels.append("mixte")    
    return labels

# --- Observed classification
observed_labels = classify_groups(groups, id_to_cluster)
observed_counts = Counter(observed_labels)
new_df2['Assembly_Cluster_ID']= observed_labels

# --- Permutation test
n_perms = 5000
null_dists = defaultdict(list)

for _ in range(n_perms):
    # Shuffle cluster labels across individuals
    shuffled_clusters = dict(zip(ids, random.sample(cluster_labels, len(cluster_labels))))
    shuffled_labels = classify_groups(groups, shuffled_clusters)
    counts = Counter(shuffled_labels)
    for label in observed_counts:
        null_dists[label].append(counts.get(label, 0))

# --- Z-scores and p-values
results = []
for label in observed_counts:
    obs = observed_counts[label]
    null = null_dists[label]
    mean = np.mean(null)
    std = np.std(null)
    z = (obs - mean) / std if std > 0 else 0
    # Two-tailed p-value
    p = (np.sum(np.abs(np.array(null) - mean) >= abs(obs - mean)) + 1) / (n_perms + 1)
    results.append((label, obs, mean, std, z, p))

# --- Output
print("\nAssembly Cluster ID Distribution Test")
print(f"{'ID':>10} {'Obs':>5} {'Mean':>7} {'Std':>7} {'Z':>6} {'p-value':>8}")
for label, obs, mean, std, z, p in sorted(results, key=lambda x: x[-1]):
    print(f"{label:>10} {obs:5} {mean:7.2f} {std:7.2f} {z:6.2f} {p:8.4f}")


df = pd.DataFrame(results, columns=["Cell_Assembly_ID", "Obs", "PermMean", "Std", "Z", "p"])
df['Obs_prop'] = df['Obs'] / df['Obs'].sum() *100
df['Perm_prop'] = df['PermMean'] / df['PermMean'].sum() *100
df["Perm_std"] = df["Std"] / df["PermMean"].sum() *100

df = df.sort_values("Cell_Assembly_ID")  # or use a custom order
plt.figure(figsize=(3, 3))
plt.errorbar(df["Cell_Assembly_ID"], df["Perm_prop"], yerr=df["Perm_std"], fmt='o', color='gray', label='Permuted ± std')
plt.scatter(df["Cell_Assembly_ID"], df["Obs_prop"], color='black', label='Observed')
plt.ylabel("Proportion")
plt.title("Observed vs Permuted Proportions")
plt.legend()
plt.tight_layout()
plt.show()

Save cell assembly ID 

In [ ]:
merged = pd.merge(combined_df0, new_df2[['Assembly_ID', 'Assembly_Cluster_ID']], on='Assembly_ID', how='outer')
merged.to_excel(f'{Path(dfilepath).parent}/CellAssembly_Global_cluster_ID.xlsx')

Activity of cell assemblies per ID

In [ ]:
merged = merged[merged['Substate'] != 'IS']
merged = merged[merged['Substate'] != 'undefined']
combined_df2 = merged.pivot_table(index='Assembly_ID', columns=[merged['Assembly_Cluster_ID'], merged['Substate']], values='Avg_Activity', aggfunc='mean', fill_value=None)
custom_order = ['AW', 'QW', 'NREM', 'REM']
sort_key = {val: i for i, val in enumerate(custom_order)}
sorted_cols = sorted(combined_df2.columns, key=lambda x: (x[0], sort_key.get(x[1], float('inf'))))
combined_df2 = combined_df2[sorted_cols]

plt.figure(figsize=(7, 3))
for idx, row in combined_df2.iterrows():
    plt.plot(row.values, label=f'{idx}', alpha=0.5, linewidth=2)  # `idx` is the MultiIndex for each row
plt.ylabel('Avg activity per cell assemblies')
plt.tight_layout()
plt.xticks(ticks= np.arange(len(combined_df2.columns.get_level_values(1))),  labels=combined_df2.columns.get_level_values(1)) 
plt.title('Cluster0                Cluster1                Cluster2                   Mixte')


combined_df2 = merged.pivot_table(index='Assembly_ID', columns=[merged['Assembly_Cluster_ID'], merged['Substate']], values='EventFreq', aggfunc='mean', fill_value=None)
custom_order = ['AW', 'QW', 'NREM', 'REM']
sort_key = {val: i for i, val in enumerate(custom_order)}
sorted_cols = sorted(combined_df2.columns, key=lambda x: (x[0], sort_key.get(x[1], float('inf'))))
combined_df2 = combined_df2[sorted_cols]

plt.figure(figsize=(7, 3))
for idx, row in combined_df2.iterrows():
    plt.plot(row.values, label=f'{idx}', alpha=0.5, linewidth=2)  # `idx` is the MultiIndex for each row
plt.ylabel('Event frequency per cell assemblies')
plt.tight_layout()
plt.xticks(ticks= np.arange(len(combined_df2.columns.get_level_values(1))),  labels=combined_df2.columns.get_level_values(1)) 
plt.title('Cluster0                Cluster1                Cluster2                   Mixte')

plt.show()